In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My\ Drive

/content/drive/My Drive


In [0]:
!sudo apt-get install wamerican

In [0]:
!cat /usr/share/dict/words > english.txt

In [0]:
!wc -l english.txt

In [0]:
cd bert

/content/drive/My Drive/bert


In [0]:
!wget -O data/data.csv https://raw.githubusercontent.com/pratikpc/AIHackathon/fcknlp/all_data.csv?token=AKBFW4B64NHDSZDUWJAUVUS5N7D7M

In [0]:
!git clone https://github.com/google-research/bert.git

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [0]:
!unzip uncased_L-12_H-768_A-12.zip

In [0]:
import pandas as pd
from sklearn.utils import shuffle
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
Data = "/content/drive/My Drive/bert/data/data.csv"
data = pd.read_csv(Data)
data['alpha'] = "a"
data.drop('Unnamed: 0', axis =1, inplace = True)
data.rename(columns = {'text': 'sentence'}, inplace = True)
data.label = data.label.astype(int)
data = shuffle(data)
data['id'] = range(len(data))

In [0]:
lmtzr = WordNetLemmatizer()

In [0]:
with open('english.txt', 'r') as f:
  text = f.readlines()
print(len(text))
eng_words_ubuntu = set([lmtzr.lemmatize(x.strip().lower().replace('\'s', '')) for x in text] )
eng_dictionary_ubuntu = dict.fromkeys(eng_words_ubuntu, None)


102305


In [0]:
len(eng_dictionary_ubuntu)

56775

In [0]:
def is_english_word(word):
    try:
        x = eng_dictionary_ubuntu[word]
        return True
    except KeyError:
        return False
print(is_english_word("buy"))


def clean_words(x):
  res = []
  wrongs = []
  split = x.split()
  for word in split:
    word = word.strip()
    word =  lmtzr.lemmatize(word)
    if is_english_word(word) :
      res.append(word)
    else:
      wrongs.append(word)
  res_str = ' '.join(res)
  
  with open("/content/abc.txt","w") as f:
    f.writelines(wrongs)
  return res_str

data['sentence'] = data['sentence'].apply(clean_words)


In [0]:
data['word_count'] = data['sentence'].apply(lambda x: len(str(x).split()))
print(data.word_count.describe())
data = data[data.word_count > 1]
data = data.drop_duplicates(subset=['sentence','label'])

In [0]:
data.word_count.describe()


count    3783.000000
mean       10.721385
std         6.614455
min         2.000000
25%         5.000000
50%         9.000000
75%        16.000000
max        70.000000
Name: word_count, dtype: float64

In [0]:
data = data[data.word_count <= 35]

In [0]:
import numpy as np
train, dev, test = np.split(data, [int(.8*len(data)), int(.9*len(data))])

In [0]:
test = test[['id', 'sentence']]

In [0]:
train.to_csv('data/train.tsv',index = False, header=True)
dev.to_csv('data/dev.tsv' ,index = False, header=True)
test.to_csv('data/test.tsv',index =False, header = True)

In [0]:
train = pd.read_csv('data/train.tsv')

In [0]:
train.head()

,sentence,label,alpha,id,word_count
0,issue solve battery change warranty concorde m...,3,a,0,10
1,let know anything assist would happy help,3,a,1,7
2,weird noise blower motor take look found filte...,4,a,2,9
3,see condition roof rail fit dealer motor car m...,3,a,3,17
4,anyone know replace mirror corolla look like c...,4,a,4,10


In [0]:
!python run_classifier.py \
--task_name=cola \
--do_train=true \
--do_eval=true \
--data_dir=./data \
--vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
--bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
--init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
--max_seq_length=64 \
--train_batch_size=32 \
--learning_rate=2e-5 \
--num_train_epochs=3.0 \
--output_dir=./bert_output/

In [0]:
Bert_base_directory = 'uncased_L-12_H-768_A-12/'
trained_classifier = './bert_output/model.ckpt-'

In [0]:
mv data/test.tsv data/test1.tsv

In [0]:
import pandas as pd
new_dataframe = pd.DataFrame(
{
    "id":[1],
    "sentence":['I would like information on purchasing a new vehicle I would like to know the details on buying a new vehicle I would like information on purchasing a new Car']
})
new_dataframe['sentence']

In [0]:
new_dataframe.to_csv('data/test.tsv',index =False, header = True)

In [0]:
!python run_classifier.py \
--task_name=cola \
--do_predict=true \
--data_dir=./data \
--vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
--bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
--init_checkpoint=./bert_output/model.ckpt-370 \
--max_seq_length=64 \
--output_dir=./bert_output/